In [ ]:
# 1. Install YOLOv8
!pip install ultralytics -q

In [ ]:
# 2. Mount Google Drive (untuk save model)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 3. Upload dataset.zip
# Upload file zip yang berisi folder: train/, val/, test/, data.yaml
from google.colab import files
print("Upload file dataset.zip...")
uploaded = files.upload()

In [ ]:
# 4. Extract dataset
!unzip -q dataset.zip -d /content/
!ls /content/data/

In [ ]:
# 5. Buat data.yaml yang sesuai path Colab
yaml_content = """path: /content/data
train: train/images
val: val/images
test: test/images

names:
  0: head_down
  1: head_up
  2: head_left
  3: head_right
  4: head_center
  5: eye_center
  6: eye_left
  7: eye_right
  8: eye_up
  9: eye_down
  10: eye_closed
  11: lip_closed
  12: lip_open

nc: 13
"""

with open('/content/data/data.yaml', 'w') as f:
    f.write(yaml_content)

print("data.yaml created!")
!cat /content/data/data.yaml

In [ ]:
# 6. Cek GPU
import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
print(f"CUDA: {torch.cuda.is_available()}")

In [ ]:
# 7. Training YOLOv8 - Optimized untuk Colab Free Tier (36 epoch limit)
from ultralytics import YOLO

# Load pretrained model
model = YOLO('yolov8n.pt')

# Training dengan strategi cepat tapi tetap akurat:
# - Epochs dikurangi ke 30 (di bawah limit 36)
# - Batch size dinaikkan untuk konvergensi lebih cepat
# - Cache=True untuk speed boost
# - Close mosaic lebih awal untuk fine-tuning
# - Warmup epochs dikurangi
results = model.train(
    data='/content/data/data.yaml',
    epochs=30,              # ✅ Di bawah limit 36, masih cukup untuk konvergen
    imgsz=640,
    batch=32,               # ✅ Naikkan dari 16->32 (GPU T4 bisa handle)
    patience=8,             # ✅ Turun dari 10->8 untuk early stopping lebih cepat
    cache=True,             # ✅ Cache images di RAM = 2-3x lebih cepat!
    project='/content/runs',
    name='cheating_detector',
    exist_ok=True,
    pretrained=True,
    optimizer='AdamW',      # ✅ AdamW sedikit lebih baik dari Adam
    lr0=0.002,              # ✅ Learning rate dinaikkan untuk konvergensi lebih cepat
    lrf=0.01,
    warmup_epochs=2,        # ✅ Kurangi dari 3->2
    warmup_momentum=0.8,
    weight_decay=0.0005,
    close_mosaic=8,         # ✅ Close mosaic lebih awal (epoch 22) untuk fine-tune
    # Augmentasi tetap aktif (penting untuk generalisasi):
    augment=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.1,
    # Multi-scale training untuk akurasi lebih baik
    amp=True,               # ✅ Automatic Mixed Precision = lebih cepat + hemat memory
    verbose=True,
)

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print(f"Total epochs: {len(results.results_dict) if hasattr(results, 'results_dict') else 30}")
print(f"Best model: /content/runs/cheating_detector/weights/best.pt")
print("="*60)


In [ ]:
# 8. Lihat hasil training
from IPython.display import Image, display

# Confusion matrix
display(Image('/content/runs/cheating_detector/confusion_matrix.png', width=600))

# Results
display(Image('/content/runs/cheating_detector/results.png', width=800))

In [ ]:
# 9. Test model
model = YOLO('/content/runs/cheating_detector/weights/best.pt')

# Validasi
metrics = model.val()
print(f"\nmAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")

In [ ]:
# 10. Copy model ke Google Drive
!mkdir -p /content/drive/MyDrive/CheatingDetector
!cp /content/runs/cheating_detector/weights/best.pt /content/drive/MyDrive/CheatingDetector/
!cp /content/runs/cheating_detector/weights/last.pt /content/drive/MyDrive/CheatingDetector/
print("\n✅ Model saved to Google Drive: /CheatingDetector/best.pt")

In [ ]:
# 11. Download model langsung
from google.colab import files
files.download('/content/runs/cheating_detector/weights/best.pt')